<a href="https://colab.research.google.com/github/vitoriacmn/IA_python/blob/main/Simula%C3%A7%C3%A3o_de_Streaming_de_Dados_no_Google_Colab_com_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


In [2]:
import json
import time

In [5]:
def gerar_dados_simulados():
    eventos = [
      {"id_transacao": 1, "status": "concluida", "valor": 100.0,
      "timestamp": "2024-12-12T12:00:00Z"}, {"id_transacao": 2, "status": "erro", "valor": 200.0,
      "timestamp": "2024-12-12T12:01:00Z"}, {"id_transacao": 3, "status": "concluida", "valor": 300.0,
      "timestamp": "2024-12-12T12:02:00Z"}
    ]
    for evento in eventos:
      print(json.dumps(evento))
      time.sleep(1) # Simulando um intervalo entre eventos
gerar_dados_simulados()


{"id_transacao": 1, "status": "concluida", "valor": 100.0, "timestamp": "2024-12-12T12:00:00Z"}
{"id_transacao": 2, "status": "erro", "valor": 200.0, "timestamp": "2024-12-12T12:01:00Z"}
{"id_transacao": 3, "status": "concluida", "valor": 300.0, "timestamp": "2024-12-12T12:02:00Z"}


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
# Configuração do Spark
spark = SparkSession.builder.appName("SimulacaoStreaming").getOrCreate()
# Dados simulados
dados = [
{"id_transacao": 1, "status": "concluida", "valor": 100.0,
"timestamp": "2024-12-12T12:00:00Z"},
{"id_transacao": 2, "status": "erro", "valor": 200.0, "timestamp":
"2024-12-12T12:01:00Z"},
{"id_transacao": 3, "status": "concluida", "valor": 300.0,
"timestamp": "2024-12-12T12:02:00Z"}
]
df = spark.createDataFrame(dados)
# Transformação: Filtrar apenas transações concluídas
df_concluidas = df.filter(col("status") == "concluida")
df_concluidas.show()


+------------+---------+--------------------+-----+
|id_transacao|   status|           timestamp|valor|
+------------+---------+--------------------+-----+
|           1|concluida|2024-12-12T12:00:00Z|100.0|
|           3|concluida|2024-12-12T12:02:00Z|300.0|
+------------+---------+--------------------+-----+



In [12]:
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
df_total = df_concluidas.agg(_sum("valor").alias("total_valor"))
df_total.show()


+-----------+
|total_valor|
+-----------+
|      400.0|
+-----------+



In [13]:
df_total.coalesce(1).write.mode("overwrite").csv("resultado_transacoes")


In [15]:
import shutil
shutil.move("/content/resultado_transacoes/part-00000-0776b548-5b20-406d-b5d1-e061de2b1296-c000.csv", "total_transacoes.csv")

'total_transacoes.csv'